In [2]:
import os
import torch
import torch.nn.functional as F
import torchmetrics
import math
from torchtext.data.metrics import bleu_score
from rouge_score import rouge_scorer

from main import get_model
from dataset.dataset import (
    LoaderConstructor,
    create_poetryfoundation_dataset,
)

In [3]:
cfg = {
    "dataset": "poetryfoundation", 
    "batch_size": 3,
    "max_length": 50,
    "embed_dim": 512,
    "min_text_length": 50, 
}

# Load the dataset
if cfg["dataset"] == "poetryfoundation":
    dataset = create_poetryfoundation_dataset(os.getcwd())
elif cfg["dataset"] == "poems_txt":
    dataset = create_poems_txt_dataset(os.getcwd())

# Construct the dataloaders
lc = LoaderConstructor(
    dataset=dataset,
    batch_size=cfg["batch_size"],
    max_length=cfg["max_length"],
    labels_sequence=False,
    min_freq=3,
)
loaders = {}
for loader in ["train", "validation", "test"]:
    loaders[loader] = lc.construct_loader(split=loader)

input_size = loaders["train"].dataset.input_size
vocab_size = lc.vocab_size
output_size = lc.output_size
device = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
def initialize_model(model_name):
    """Loads the model with trained weights"""
    model = get_model(
        model=model_name,
        vocab_size=vocab_size,
        embed_dim=cfg["embed_dim"],
        seq_len=input_size,
        output_dim=output_size,
        device=device,
    )

    model_weights = {
        "lstm": f"trained_models/lstm_{cfg['dataset']}_lr=0_001_best.pt", 
        "transformer": f"trained_models/transformer_{cfg['dataset']}_lr=0_0001_best.pt",
    }

    model.load_state_dict(torch.load(model_weights[model_name]))
    model.to(device).eval()
    return model


In [5]:
criterion = torch.nn.CrossEntropyLoss()
accuracy_metric = torchmetrics.Accuracy(task="multiclass", num_classes=output_size).to(device)
top3_accuracy_metric = torchmetrics.Accuracy(task="multiclass", num_classes=output_size, top_k=5).to(device)

def calculate_perplexity(loss):
    return math.exp(loss)

def calculate_bleu(candidate, references):
    return bleu_score([candidate], [[ref] for ref in references])

def calculate_rouge(candidate, reference):
    scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)
    scores = scorer.score(candidate, reference)
    return scores

def evaluate(model, batch, test=False, n_next_words=5):
    """Evaluates the model and predicts next words for poetry"""
    inputs, labels = batch["input_ids"].to(device), batch["labels"].to(device)
    labels = labels.contiguous().view(-1)
    output = model(inputs).view(-1, output_size)

    loss = criterion(output, labels).item()
    accuracy = accuracy_metric(torch.argmax(output, dim=-1), labels).item()
    perplexity = calculate_perplexity(loss)
    top3_accuracy = top3_accuracy_metric(output, labels).item()

    print(f"Loss: {loss:.4f}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Perplexity: {perplexity:.4f}")
    print(f"Top-3 Accuracy: {top3_accuracy:.4f}")

    total_bleu = 0
    total_rouge = {"rouge1": 0, "rouge2": 0, "rougeL": 0}
    num_samples = batch["input_ids"].shape[0]

    for i in range(num_samples):
        input_text = lc.tokenizer.decode(inputs[i].tolist(), target=False)
        target_text = lc.tokenizer.decode(labels[i].unsqueeze(0).tolist(), target=True)[0]
        predicted_text = lc.tokenizer.decode(torch.argmax(output[i], dim=-1).unsqueeze(0).tolist(), target=True)[0]

        print(f"Input: {' '.join(input_text)}")
        print(f"Target: {target_text}")
        print(f"Predicted: {predicted_text}")
        print()

        bleu = calculate_bleu(predicted_text.split(), [target_text.split()])
        total_bleu += bleu

        rouge = calculate_rouge(predicted_text, target_text)
        for key in total_rouge.keys():
            total_rouge[key] += rouge[key].fmeasure

    avg_bleu = total_bleu / num_samples
    avg_rouge = {key: total_rouge[key] / num_samples for key in total_rouge}

    print(f"Avg BLEU Score: {avg_bleu}")
    print(f"Avg ROUGE Score: {avg_rouge}")

    # Predict next words for new poetic lines
    if test:
        mini_dataset = [
            {"text": "The moonlight danced upon the silent lake", "predictions": []},
            {"text": "A whisper of wind through the autumn leaves", "predictions": []},
            {"text": "The poet's heart was filled with endless wonder", "predictions": []},
            {"text": "Beneath the stars, she walked alone", "predictions": []},
            {"text": "His voice was music, soft and deep", "predictions": []},
        ]

        # Tokenize the texts
        tokenised_samples = lc.tokenizer.create_tokens(mini_dataset)
        tokenised_samples = lc.tokenizer.pad_sequences(tokenised_samples)
        encodings = lc.tokenizer.encode(tokenised_samples)

        # No labels in this case
        inputs = encodings[:, 1:].to(device)
        decoded_inputs = [lc.tokenizer.decode(sample.tolist(), target=False) for sample in inputs]

        with torch.no_grad():
            for _ in range(n_next_words):
                output = model(inputs)
                next_token = torch.argmax(output, dim=-1)

                # Decode the tokens to get the words
                decoded_tokens = lc.tokenizer.decode(next_token.tolist(), target=True)

                # Encode the tokens to get the input for the next iteration
                encoded_tokens = lc.tokenizer.encode([decoded_tokens]).to(device)

                # Concatenate the input with the new tokens
                inputs = torch.cat([inputs[:, 1:], encoded_tokens.reshape(-1, 1)], dim=1)

                for i, token in enumerate(decoded_tokens):
                    mini_dataset[i]["predictions"].append(token)

        for sample, input_sentence in zip(mini_dataset, decoded_inputs):
            print(f"Input: {' '.join(input_sentence)}")
            print(f"Predicted next words: {' '.join(sample['predictions'])}")
            print()


In [6]:
# **Run Evaluation**
batch = next(iter(loaders["train"]))
for model_name in ["lstm", "transformer"]:
    print("*" * 50)
    print(f"Evaluating model: {model_name}")
    print("*" * 50)
    model = initialize_model(model_name)
    evaluate(model, batch)
    print("*" * 50)


**************************************************
Evaluating model: lstm
**************************************************
Loss: 5.5677
Accuracy: 0.0000
Perplexity: 261.8204
Top-3 Accuracy: 0.0000
Input: <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> the
Target: rainbow
Predicted: hermit

Input: <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <oov>
Target: hymn
Predicted: song

Input: <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pa

In [8]:
batch = next(iter(loaders["test"]))
for model_name in ["lstm", "transformer"]:
    print("*" * 50)
    print(f"Evaluating model: {model_name}")
    print("*" * 50)
    model = initialize_model(model_name)
    evaluate(model, batch, test=True, n_next_words=3)
    print("*" * 50)

**************************************************
Evaluating model: lstm
**************************************************
Loss: 7.3273
Accuracy: 0.0000
Perplexity: 1521.3400
Top-3 Accuracy: 0.0000
Input: <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> woman <oov> ocean on a <oov> through a glass <oov>
Target: boat
Predicted: me

Input: <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> battle hymn of the
Target: republic
Predicted: world

Input: <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad

Loss: 7.0914
Accuracy: 0.0000
Perplexity: 1201.6148
Top-3 Accuracy: 0.0000
Input: <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> woman <oov> ocean on a <oov> through a glass <oov>
Target: boat
Predicted: me

Input: <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> battle hymn of the
Target: republic
Predicted: world

Input: <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pa